In [1]:
# Colab PRO一個月300有找，就有V100用，好耶
!nvidia-smi

Fri Jul 30 07:50:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# 混合精度加速訓練
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

**Download Data**

In [4]:
!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!unzip data.zip
!ls 

Downloading...
From: https://drive.google.com/uc?id=1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR
To: /content/data.zip
372MB [00:02, 164MB/s]
Archive:  data.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
data.zip  sample_data  timit_11


**Preparing Data**

In [5]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy').reshape(-1, 11, 39)
train_label = np.load(data_root + 'train_label_11.npy').astype('int')
test = np.load(data_root + 'test_11.npy').reshape(-1, 11, 39)
print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 11, 39)
Size of testing data: (451552, 11, 39)


In [6]:
import tensorflow as tf
from tqdm.notebook import tqdm

def data_to_sents(data, label=None):
  sent, sent_label = [], []
  sents, sent_labels = [], []

  for i in tqdm(range(len(data))):
    if i == 0:
      sent = list(data[i])
      if label is not None:
        sent_label = [label[i]]
      continue

    if np.any(data[i][0] != data[i-1][1]):
      sents.append(np.array(sent))
      sent = list(data[i])
            
      if label is not None:
        sent_labels.append(sent_label)
        sent_label = [label[i]]
                
    else:
      sent.append(data[i][-1])
      if label is not None:
        sent_label.append(label[i])

  sents.append(np.array(sent))
  if label is not None:
    sent_labels.append(sent_label)
    
  if label is not None:    
    for i in range(len(sent_labels)):
      sent_labels[i] = [sent_labels[i][0]]*5 + sent_labels[i] + [sent_labels[i][-1]]*5  
    return sents, sent_labels 
 
  return sents

train_sents, train_sent_labels = data_to_sents(train, train_label)
test_sents = data_to_sents(test)

In [7]:
# 將資料補到相同長度
padded_train_sents = tf.keras.preprocessing.sequence.pad_sequences(
  train_sents, maxlen=725, padding="post", dtype='float32', value=0.0
)
padded_train_labels = tf.keras.preprocessing.sequence.pad_sequences(
  train_sent_labels, maxlen=725, padding="post", dtype='int32', value=-1
)
padded_test_sents = tf.keras.preprocessing.sequence.pad_sequences(
  test_sents, maxlen=725, padding="post", dtype='float32', value=0.0
)
# Sample weights，沒有資料的部分不要算loss
padded_train_labels_sw = (padded_train_labels != -1).astype('int')
padded_train_labels[padded_train_labels == -1] = 0
padded_train_sents.shape, padded_train_labels.shape, padded_test_sents.shape

((4620, 725, 39), (4620, 725), (1680, 725, 39))

**Create Model**

In [8]:
from tensorflow.keras.layers import Bidirectional, LSTM, GRU, Dense, Conv1D
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten, GlobalAveragePooling1D, Dropout
from tensorflow.keras.layers import Conv2D, Reshape, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import tensorflow.keras.backend as K
import tensorflow.keras as keras

def make_model():
  model = keras.Sequential([
    keras.Input(shape = (725, 39)),
    keras.layers.Masking(mask_value = 0.0),
    Bidirectional(LSTM(256, return_sequences = True)),
    Dropout(0.25),
    Bidirectional(LSTM(256, return_sequences = True)),
    Dropout(0.25),
    Bidirectional(LSTM(256, return_sequences = True)),
    Dropout(0.25),
    Dense(39, activation = 'softmax'),
  ])
  model.compile(RMSprop(learning_rate = 0.01), 'sparse_categorical_crossentropy', 'accuracy')
  return model

In [9]:
from sklearn.model_selection import KFold, StratifiedKFold

k_folder = KFold(n_splits = 20)
for i_fold, (train_index, valid_index) in enumerate(tqdm(k_folder.split(padded_train_sents))):
  tf.random.set_seed(8889 + i_fold)
  x_train, x_valid = padded_train_sents[train_index], padded_train_sents[valid_index]
  y_train, y_valid = padded_train_labels[train_index], padded_train_labels[valid_index]
  sw_train, sw_valid = padded_train_labels_sw[train_index], padded_train_labels_sw[valid_index]

  model = make_model()
  early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 4, restore_best_weights = True, verbose = 1)
  lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 1, verbose = 1, min_lr = 5e-4, min_delta = 0.0003)

  history = model.fit(x_train, y_train, sample_weight = sw_train, epochs = 100, batch_size = 64,
             validation_batch_size = 256, 
             validation_data = (x_valid, y_valid, sw_valid),
             callbacks = [early_stop, lr_schedule])
  model.save(f'20210729.h5')

Epoch 1/100
69/69 [==============================] - 48s 410ms/step - loss: 1.2300 - accuracy: 0.1739 - val_loss: 0.9321 - val_accuracy: 0.2895
Epoch 2/100
69/69 [==============================] - 22s 323ms/step - loss: 0.7393 - accuracy: 0.4299 - val_loss: 0.5850 - val_accuracy: 0.5463
Epoch 3/100
69/69 [==============================] - 22s 325ms/step - loss: 0.4832 - accuracy: 0.6139 - val_loss: 0.3953 - val_accuracy: 0.6777
Epoch 4/100
69/69 [==============================] - 22s 325ms/step - loss: 0.3771 - accuracy: 0.6902 - val_loss: 0.4143 - val_accuracy: 0.6608

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.
Epoch 5/100
69/69 [==============================] - 22s 323ms/step - loss: 0.2756 - accuracy: 0.7665 - val_loss: 0.3105 - val_accuracy: 0.7404
Epoch 6/100
69/69 [==============================] - 22s 325ms/step - loss: 0.2354 - accuracy: 0.7964 - val_loss: 0.3044 - val_accuracy: 0.7473
Epoch 7/100
69/69 [==============================] - 22s

In [10]:
model_pred = []
for i_fold in range(20):
  model = keras.models.load_model(f'20210729.h5')
  test_pred = model.predict(padded_test_sents, batch_size = 256, verbose = 1)
  model_pred.append(test_pred)

7/7 [==============================] - 7s 268ms/step


In [ ]:
test_pred = np.mean(model_pred, axis=0)
print(test_pred)

In [12]:
# 將資料格式還原
test_pred_idx = np.argmax(test_pred, axis=2)
test_pred_flat = []
for pred, test_sent in zip(test_pred_idx, test_sents):
  test_pred_flat.extend(pred[5:5+len(test_sent)-10])

In [13]:
import pandas as pd

submission = pd.DataFrame()
submission['Class'] = test_pred_flat
submission.to_csv('20210729.csv', index_label='Id')